In [1]:
# =====================================================
# 1. Imports & Setup
# =====================================================
import pandas as pd
import re
import os

In [2]:
# =====================================================
# 2. Load Raw Dataset
# =====================================================
print("📂 Loading raw dataset...")
df = pd.read_csv("data/raw_tours.csv")
print(f"✅ Loaded {df.shape[0]} rows and {df.shape[1]} columns")

📂 Loading raw dataset...
✅ Loaded 20 rows and 11 columns


In [3]:
# =====================================================
# 3. Initial Exploration
# =====================================================

In [4]:
# ---------------------------------------
# View first 5 rows
# ---------------------------------------
df.head()

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]


In [5]:
# ---------------------------------------
# Get basic info
# ---------------------------------------
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Rank                           20 non-null     int64 
 1   Peak                           9 non-null      object
 2   All Time Peak                  6 non-null      object
 3   Actual gross                   20 non-null     object
 4   Actual gross(in 2022 dollars)  20 non-null     object
 5   Artist                         20 non-null     object
 6   Tour title                     20 non-null     object
 7   Year(s)                        20 non-null     object
 8   Shows                          20 non-null     int64 
 9   Average gross                  20 non-null     object
 10  Ref.                           20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


In [6]:
# ---------------------------------------
# View missing values
# ---------------------------------------
df.isnull().sum()

Rank                              0
Peak                             11
All Time Peak                    14
Actual gross                      0
Actual gross(in 2022 dollars)     0
Artist                            0
Tour title                        0
Year(s)                           0
Shows                             0
Average gross                     0
Ref.                              0
dtype: int64

In [7]:
# ---------------------------------------
# Summary statistics
# ---------------------------------------
df.describe(include='all')

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
count,20.000000,9,6,20,20,20,20,20,20.000000,20,20
unique,NaN,7,6,20,20,9,20,16,NaN,20,20
top,NaN,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2013–2014,NaN,"$13,928,571",[1]
freq,NaN,2,1,1,1,4,1,3,NaN,1,1
mean,10.450000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.000000,NaN,NaN
std,5.942488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.507617,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.000000,NaN,NaN
25%,5.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.000000,NaN,NaN
50%,10.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.000000,NaN,NaN
75%,15.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.500000,NaN,NaN


In [8]:
# =====================================================
# 4. Cleaning Functions
# =====================================================

In [9]:
# ---------------------------------------
# Clean currency columns
# ---------------------------------------
def clean_currency_column(df, column_name):
    df[column_name] = (
        df[column_name]
        .astype(str)
        .str.replace(r'[\$,]', '', regex=True)           # Remove $ and commas
        .str.replace(r'\[.*?\]', '', regex=True)         # Remove footnote brackets like [b], [a], etc.
        .str.strip()
    )

    # Replace empty strings with 0, then convert to float
    df[column_name] = df[column_name].replace('', '0').astype(float)

    return df

In [10]:
# ---------------------------------------
# Clean text columns (remove symbols/footnotes)
# ---------------------------------------
def clean_text_column(df, column_name):
    df[column_name] = (
        df[column_name]
        .astype(str)
        .apply(lambda x: re.sub(r'\[.*?\]|†|‡', '', x).strip())
    )
    return df

In [11]:
# ---------------------------------------
# Split year ranges
# ---------------------------------------
def split_year_range(df):
    df[['Start Year', 'End Year']] = df['Year(s)'].str.extract(r'(\d{4})[–-](\d{4})')
    df['Start Year'] = df['Start Year'].astype(float)
    df['End Year'] = df['End Year'].astype(float)
    return df

In [12]:
# ---------------------------------------
# Fill missing peak values
# ---------------------------------------
def fill_missing_peaks(df, column_name, fill_value='Unknown'):
    df[column_name] = df[column_name].fillna(fill_value)
    return df

In [13]:
# =====================================================
# 5. Apply Cleaning
# =====================================================

In [14]:
print("💰 Cleaning currency columns...")
df = clean_currency_column(df, 'Actual gross')
df = clean_currency_column(df, 'Actual gross(in 2022 dollars)')
df = clean_currency_column(df, 'Average gross')

💰 Cleaning currency columns...


In [15]:
print("🧼 Cleaning text columns...")
df = clean_text_column(df, 'Tour title')
df = clean_text_column(df, 'Artist')
df = clean_text_column(df, 'Peak')
df = clean_text_column(df, 'All Time Peak')

🧼 Cleaning text columns...


In [16]:
print("📆 Splitting year ranges...")
df = split_year_range(df)

📆 Splitting year ranges...


In [17]:
print("📈 Filling missing peaks...")
df = fill_missing_peaks(df, 'Peak')
df = fill_missing_peaks(df, 'All Time Peak')

📈 Filling missing peaks...


In [18]:

# ---------------------------------------
# Drop unnecessary column
# ---------------------------------------
if 'Ref.' in df.columns:
    df.drop(columns=['Ref.'], inplace=True)

In [19]:
# =====================================================
# 6. Export Cleaned Dataset
# =====================================================
if not os.path.exists('cleaned'):
    os.makedirs('cleaned')

In [20]:
df.to_csv("cleaned/cleaned_tours.csv", index=False)
print("✅ Cleaned dataset saved to cleaned/cleaned_tours.csv")

✅ Cleaned dataset saved to cleaned/cleaned_tours.csv
